In [1]:
from pronouncing import phones_for_word, syllable_count
from collections import defaultdict
from functions import *
import json
import os
import random
import wordninja
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import PlaintextCorpusReader
from nltk.corpus import stopwords

# miscellany
from collections import Counter
import time
import gzip
import pickle

# reload functions/libraries when edited
%load_ext autoreload
%autoreload 2

In [2]:
pronunciation_list = phones_for_word('programming')
syllable_count(pronunciation_list[0])

3

In [3]:
pronunciation_list

['P R OW1 G R AE2 M IH0 NG']

In [4]:
type(pronunciation_list)

list

In [5]:
syllable_count(phones_for_word('enumerate')[0])

4

In [6]:
file_path = os.path.expanduser('~/Documents/python_poetry/poetrydb_poem_generator/')
f = open(file_path + 'poems_raw.txt', 'r')
poems_raw = f.read()
poems_raw[:100]

'where shall we go for our garlands glad \n at the falling of the year \n when the burntup banks are ye'

In [24]:
poems_segmented = wordninja.split(poems_raw)
poems_segmented[15:20]

['the', 'burnt', 'up', 'banks', 'are']

In [26]:
poems_segmented[:10]

['where', 'shall', 'we', 'go', 'for', 'our', 'garlands', 'glad', 'at', 'the']

In [41]:
'maries' in poems_segmented

True

In [46]:
poems_known_words = [word for word in poems_segmented if phones_for_word(word)]

In [47]:
'maries' in poems_known_words

False

In [53]:
poems_dictionary = defaultdict(list)

for current_word, next_word in zip(poems_known_words, poems_known_words[1:]):
    poems_dictionary[current_word].append(next_word)

poems_dictionary['land'][:5]

['must', 'and', 'shows', 'where', 'while']

In [54]:
'maries' in poems_dictionary

False

In [67]:
def tankanizer(po_dict):

    tanka = []
    line_1 = []
    line_2 = []
    line_3 = []
    line_4 = []
    line_5 = []
    
    syllables = 0
    while syllables == 0:
        first_word = random.choice(list(po_dict.keys()))
        syl_count = syllable_count(phones_for_word(first_word)[0])
        if syl_count <= 5:
            line_1.append(first_word)
            syllables += syl_count
        else:
            continue
    
    while syllables < 5:
        next_word = random.choice(po_dict[line_1[-1]])
        syl_count = syllable_count(phones_for_word(next_word)[0])
        if syl_count <= (5 - syllables):
            line_1.append(next_word)
            syllables += syl_count
        else:
            continue
    
    print((' ').join(line_1))

In [2]:
os.path.expanduser("~/Documents/python_poetry/custom_corpus_streams/writings")

'/Users/p.szymo/Documents/python_poetry/custom_corpus_streams/writings'

In [8]:
writings_path = os.path.expanduser("~/Documents/python_poetry/custom_corpus_streams/writings")

corpus = PlaintextCorpusReader(writings_path, '.*', word_tokenizer=RegexpTokenizer(pattern="([a-zA-Z]+(?:'[a-z]+)?)"))

corpus.words()[:20]

['the',
 'world',
 'knew',
 'it',
 'was',
 'december',
 'my',
 'fucking',
 'chest',
 'is',
 'caving',
 'in',
 'teeth',
 'all',
 'grimy',
 'and',
 'shit',
 'seventeen',
 'graffitto',
 "ain't"]

In [12]:
corpus_words = [word.lower() for word in corpus.words() if phones_for_word(word)]
corpus_words[:20]

['the',
 'world',
 'knew',
 'it',
 'was',
 'december',
 'my',
 'fucking',
 'chest',
 'is',
 'caving',
 'in',
 'teeth',
 'all',
 'grimy',
 'and',
 'shit',
 'seventeen',
 "ain't",
 'got']

In [13]:
words = []
for word in corpus_words:
    if len(word) != 1:
        words.append(word)
    else:
        if word in ['a', 'e', 'i', 'o', 'u', 'x', 'y', 'z']:
            words.append(word)

In [14]:
words_dictionary = defaultdict(list)

for current_word, next_word in zip(words, words[1:]):
    words_dictionary[current_word].append(next_word)

words_dictionary['the'][:5]

['world', 'mark', 'willows', 'twain', 'house']

In [18]:
for i in range(5):
    tankanizer(words_dictionary)
    print('\n\n\n')

boroughs oh wait i




noisy creak i ve




worth in black snow well




surface beneath the




hydrated to tell






In [83]:
line = []
count = 0
while count == 0:
    start_word = random.choice(list(words_dictionary.keys()))
    count = syllable_count(phones_for_word(start_word)[0])
    if count <= 5:
        line.append(start_word)
    else:
        count = 0
        continue

In [84]:
line

['sprout']

In [80]:
start_word

'trains'

In [81]:
words_dictionary[start_word]

['of', 'tossed']

In [101]:
def line_creator(word_dict, num_syllables, prompt=None):
    '''Input a dictionary in the format of key = current word, value = list of next words
       along with the number of words you would like to see in your generated sentence.'''
    
    line = []
    count = 0
    
    if prompt:
        while count == 0:
            start_word = random.choice(word_dict[prompt])
            count = syllable_count(phones_for_word(start_word)[0])
            if count <= num_syllables:
                line.append(start_word)
            else:
                count = 0
                continue
    else:
        while count == 0:
            start_word = random.choice(list(word_dict.keys()))
            count = syllable_count(phones_for_word(start_word)[0])
            if count <= num_syllables:
                line.append(start_word)
            else:
                count = 0
                continue

    while count < num_syllables:
        next_word = random.choice(word_dict[line[-1]])
        temp_count = count + syllable_count(phones_for_word(next_word)[0])
        if temp_count <= num_syllables:
            line.append(next_word)
            count += syllable_count(phones_for_word(next_word)[0])
        else:
            continue
            
    return line

In [102]:
def tankanizer(word_dict, start_word=None):

    line_1 = line_creator(word_dict, 5, start_word)
    line_2 = line_creator(word_dict, 7, line_1[-1])
    line_3 = line_creator(word_dict, 5, line_2[-1])
    line_4 = line_creator(word_dict, 7, line_3[-1])
    line_5 = line_creator(word_dict, 7, line_4[-1])
    lines = [line_1, line_2, line_3, line_4, line_5]
    tanka = [(' ').join(line) for line in lines]
    
    print(('\n').join(tanka))

In [103]:
for i in range(5):
    print(line_creator(words_dictionary, 5))

['downing', 'a', 'program']
['identifying']
['sly', 'for', 'a', 'i', 'should']
['consciousness', 'for', 'all']
['triggers', 'Three', 'Scene', 'As']


In [20]:
print(tankanizer(words_dictionary))

thusly we got no
longer tell him for who ve
missed call let er go
away and anyway it
means stillness is nigh now to


In [6]:
# uncomment to load
with open('data/own_corpus.txt') as hello:
    text = hello.read()

In [8]:
text[:500]

'April 17, 2020\n\n10 minutes # 13\n\n\n\nyou trust the limp in your step and fetch a loose post from the earth’s grip throw it in the back of the wagon and drive on down to the place you used to call home now known for its unmown lawn its seeding grass propagating the cracks in the driveway the fresh locks shiny next to the grimed siding it seemed the perfect tool to bust in and grab that which remained the one thing you’d forgotten in the one place you hadn’t thought to look the false wall beside wit'

In [9]:
# substitute endline characters
text_edit = re.sub(r'\n', ' xnewlinex ', text)

# substitute tab characters
text_edit = re.sub(r' \t ', ' xtabx ', text_edit)

# check out a sample
text_edit[:500]

'April 17, 2020 xnewlinex  xnewlinex 10 minutes # 13 xnewlinex  xnewlinex  xnewlinex  xnewlinex you trust the limp in your step and fetch a loose post from the earth’s grip throw it in the back of the wagon and drive on down to the place you used to call home now known for its unmown lawn its seeding grass propagating the cracks in the driveway the fresh locks shiny next to the grimed siding it seemed the perfect tool to bust in and grab that which remained the one thing you’d forgotten in the on'

In [12]:
word_tokenize(text_edit)[:50]

['April',
 '17',
 ',',
 '2020',
 'xnewlinex',
 'xnewlinex',
 '10',
 'minutes',
 '#',
 '13',
 'xnewlinex',
 'xnewlinex',
 'xnewlinex',
 'xnewlinex',
 'you',
 'trust',
 'the',
 'limp',
 'in',
 'your',
 'step',
 'and',
 'fetch',
 'a',
 'loose',
 'post',
 'from',
 'the',
 'earth',
 '’',
 's',
 'grip',
 'throw',
 'it',
 'in',
 'the',
 'back',
 'of',
 'the',
 'wagon',
 'and',
 'drive',
 'on',
 'down',
 'to',
 'the',
 'place',
 'you',
 'used',
 'to']

In [34]:
tokenizer = RegexpTokenizer(pattern="[a-zA-Z’'-]+")

In [42]:
corpus_words = [word.lower() for word in tokenizer.tokenize(text)]
corpus_words[:10]

['april',
 'minutes',
 'you',
 'trust',
 'the',
 'limp',
 'in',
 'your',
 'step',
 'and']

In [69]:
%time syllable_count('z')

CPU times: user 10 µs, sys: 0 ns, total: 10 µs
Wall time: 11.9 µs


0

In [88]:
# instantiate a dictionary
text_dictionary = defaultdict(list)

# create Markov dictionary
for current_word, next_word in zip(corpus_words, corpus_words[1:]):
    text_dictionary[current_word].append(next_word)

In [116]:
# instantiate a dictionary
text_dictionary = defaultdict(list)

# create Markov dictionary
for current_word, next_word in zip(corpus_words, corpus_words[1:]):
    if phones_for_word(next_word):
        text_dictionary[current_word].append(next_word)
    else:
        text_dictionary[current_word].append('the')

In [108]:
line_creator(text_dictionary, 5, prompt='promptuescent')

['corner', 'eyes', 'the', 'hues']

In [74]:
print(tankanizer(text_dictionary))

tub they have to bust
out quick succession of the
brain juices in their
wicked sperm baby jesus
cream you because it's new york


In [109]:
print(tankanizer(text_dictionary))

into else
judges they found a
part of life struggle without
me happy to feel
my cave ate the lamb designs
a little fanny blender


In [77]:
%%time

for k,v in text_dictionary.items():
    for i,word in enumerate(v):
        if syllable_count(word):
            continue
        else:
            v[i] = 'the'

CPU times: user 78.5 ms, sys: 1.59 ms, total: 80.1 ms
Wall time: 80.2 ms


In [80]:
syllable_count(['the'])

TypeError: expected string or bytes-like object

In [110]:
{key: value for key,value in text_dictionary.items() if len(value) < 5}

{'trust': ['the', 'anybody', 'him', 'be'],
 'limp': ['in', 'dreamy', 'wrist'],
 'step': ['and', 'on', 'out', 'onto'],
 'fetch': ['a', 'a'],
 'post': ['from'],
 'earth’s': ['grip'],
 'wagon': ['and'],
 'drive': ['on', 'until', 'with'],
 'known': ['for', 'trickle', 'from'],
 'unmown': ['lawn'],
 'lawn': ['its'],
 'seeding': ['grass'],
 'propagating': ['the'],
 'cracks': ['in', 'up'],
 'driveway': ['the', 'i'],
 'fresh': ['locks', 'bow'],
 'locks': ['shiny', 'must'],
 'grimed': ['siding'],
 'siding': ['it'],
 'tool': ['to'],
 'bust': ['in', 'through', 'out', 'it'],
 'remained': ['the', 'this', 'somewhere'],
 'you’d': ['forgotten', 'never', 'live', 'like'],
 'hadn’t': ['thought', 'been'],
 'false': ['wall', 'eyelashes'],
 'drawer': ['in'],
 'row': ['the'],
 'anywayyyy': ['the'],
 'my’s': ['well'],
 'feelings': ['even', 'so', 'toward', 'of'],
 'deadended': ['the'],
 'deadenced': ['the'],
 'deadened': ['myself'],
 'borekas': ['and', 'but'],
 'slown': ['down', 'was'],
 'huh': ['i', 'need', 'i

In [87]:
{key: value for key,value in text_dictionary.items() if len(value) < 2}

{'post': ['from'],
 'earth’s': ['grip'],
 'wagon': ['and'],
 'unmown': ['lawn'],
 'lawn': ['its'],
 'seeding': ['grass'],
 'propagating': ['the'],
 'grimed': ['siding'],
 'siding': ['it'],
 'tool': ['to'],
 'drawer': ['in'],
 'row': ['the'],
 'anywayyyy': ['the'],
 'my’s': ['well'],
 'deadended': ['the'],
 'deadenced': ['the'],
 'deadened': ['myself'],
 'in’t': ['fluck'],
 'fluck': ['me'],
 'dunno': ['i'],
 'stubbed': ['toe'],
 'neckspine': ['has'],
 'shifted': ['to'],
 'employed': ['by'],
 'unassuming': ['unaware'],
 'unaware': ['mind'],
 'ultrafocused': ['on'],
 'wellbeing': ['the'],
 'ahhhhhhhhhh': ['crazy'],
 'marionetted': ['through'],
 'bloodflow': ['yeah'],
 'ourselves': ['bow'],
 'overlord': ['there'],
 'safety': ['is'],
 'guaranteed': ['in'],
 'alluring': ['that'],
 'pleasurable': ['terror'],
 'terror': ['her'],
 'guessing': ['the'],
 'kanazawa': ['i'],
 'trickling': ['flow'],
 'tasted': ['as'],
 'streamed': ['in'],
 'maximum': ['velocity'],
 'velocity': ['heated'],
 'leagues'

In [66]:
len(text_dictionary['the']), len(text_dictionary['a'])

(1962, 1058)

In [112]:
len(text_dictionary['the']), len(text_dictionary['a'])

(1962, 1058)

In [54]:
def value_tester(values):
    valid = False
    for word in values:
        if phones_for_word(word):
            valid = True
            break
        else:
            continue
    return valid

In [58]:
%%time

value_tester(text_dictionary['wellbeing'])

CPU times: user 8 µs, sys: 0 ns, total: 8 µs
Wall time: 10 µs


False

In [113]:
for i in range(10):
    print(tankanizer(text_dictionary))
    print('\n\n')

discovered the the
wave as quickly scattering
air around could on
asphalt the mantra here here
we leave the highway into



croak for those eyes now
i am how many hats the
waist the bridge taunts me
my the craziest weather
anchors probably the e



funky the door i
so at all away to big
eyes closer than five
billion so just created
by an earnestness on the



corpse with it still on
cream is theirs silence and three
dollars an eye with
you can really forked me your
book you hate me dead let me



organ dirt in the
title above it does a
key an empty or
sad i am it my legs he
was striped like the time keeping



reckoned i love you
tonight drunk we can breathe they
are you were too much
pleasure nor for those two and
silk robe he did mean by a



shower curtain bounce
the floor apartment with a
mona lisa gaze
away the crux a sort of
my wireless the the stillness





KeyboardInterrupt: 

In [117]:
%%time

good_lists = 0
bad_lists = 0
for k,v in text_dictionary.items():
    if value_tester(v):
        good_lists += 1
    else:
        print(k, v)
        bad_lists += 1

good_lists, bad_lists

CPU times: user 7.45 ms, sys: 321 µs, total: 7.77 ms
Wall time: 7.52 ms


(7051, 0)

In [106]:
syllable_count(phones_for_word('the')[0])

1